# AIM

AIM of this Notebook is to collect the outputs from 
1. base_model (LLAMA 3B)
2. Model_1 (LLAMA 3B Model fine-tuned with TL;DR and Custom Dataset)
3. Model_2 (LLAMA 3B Model Fine-tuned with Custom Dataset)

which would be later be used for its [evalution](https://github.com/au-nlp/project-milestone-p2-group-6/blob/main/lab/model_evaluation.ipynb)

In [ ]:
!unzip fine_tuned_with_cs.zip

In [ ]:
!unzip final-summary.zip

In [ ]:
!pip install pandas datasets

In [ ]:
!pip install transformers torch

In [ ]:
!pip install xformers trl peft accelerate bitsandbytes

In [ ]:
import sys
from pathlib import Path

project_root = Path.cwd().parent  # or Path().resolve().parent
sys.path.insert(0, str(project_root))
# we are doing this so we can import src folder

import json
from gc import collect
from src.utils.torch import ensure_device
from src.load_dataset import load_jsonl, CS_JSON, split_90_and_10
from src.load_model import load_tokenizer, load_model, lora_config_for
from src.extract_from import msg_for_base_model, non_assistant_messages
from src.train_model import EXPORT_CS_FINE_TUNED, EXPORT_TLDR_CS_FINE_TUNED
from src.eval_model import linearly_infer_from, batch_infer_from, EXPORT_CS_RESULTS, EXPORT_BASE_RESULTS, \
    EXPORT_CS_TLDR_RESULTS

In [7]:
ensure_device()

We would be using this device: cuda


In [8]:
# Load JSONL data (Custom Dataset)

custom_dataset = load_jsonl(CS_JSON)
val_dataset = split_90_and_10(custom_dataset)["test"]
print(f"✓ Loaded {len(val_dataset)} examples")


2-[1/8] Loading dataset...
✓ Loaded 101 examples


## Converting the Samples

Every Sample in the JSONL has three messages (system instruction, user message and then the assistant response)
and since we wanted to collect the assistant responses from models we have, we would only extract system instruction and user message from custom dataset.

we make sure to make the instructions clear for the base model as it was not fine-tuned before.

In [10]:
base_generation_inputs = val_dataset.map(msg_for_base_model)

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

## Note

we have log-in inside hugging face so we can access [Llama-3.2-3B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct) as it's a gated repo.

In [13]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `YTA-DEV` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/hu

# Model_0 Results

Results from the Base Model (LLMA 3.2 3B Instruct)

In [ ]:
base_tokenizer = load_tokenizer()
base_model = load_model()

In [ ]:
base_outputs = linearly_infer_from(base_model, base_tokenizer, base_generation_inputs)

### Exporting

we would now export the results list of (prompt_message, assistant response) to json file

we have observed with T4 GPU it took ~1.5 hours just for inference we would be trying out with higher GPU and inferencing in batches in next batch.

In [ ]:
Path(EXPORT_BASE_RESULTS).write_text(json.dumps(base_outputs))

In [ ]:
del base_model
del base_tokenizer
collect()

# we are doing this to make sure the python's garbage collector collects previous model and tokenizer to save gpu ram

# Model_1 Results

Results from the Model (LLMA 3.2 3B Instruct) which was Fine-tuned with only the Custom Dataset

In [16]:
generation_inputs = val_dataset.map(non_assistant_messages)

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [17]:
cs_tokenizer = load_tokenizer(EXPORT_CS_FINE_TUNED)

cs_model = load_model(EXPORT_CS_FINE_TUNED)
cs_model = lora_config_for(cs_model, EXPORT_CS_FINE_TUNED, for_training=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

### Note

from the previous run we have observed th

In [19]:
cs_outputs = batch_infer_from(cs_model, cs_tokenizer, generation_inputs, batch=4)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


3.96% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


7.92% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

87.13% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


91.09% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


95.05% complete
99.01% complete
100.00% complete


### Exporting

we would now export the results list of (prompt_message, assistant response) to json file

we have observed with A100 GPU it took ~1 hour for inference since we have tried to utilize more GPU (with batch: 4)

In [ ]:
Path(EXPORT_CS_RESULTS).write_text(json.dumps(cs_outputs))

In [21]:
del cs_model
del cs_tokenizer
collect()

12296

# Model_2 Results

Results from the Model which was Fine-Tuned with the TL;DR and then Custom Dataset

In [31]:
cts_tokenizer = load_tokenizer(EXPORT_TLDR_CS_FINE_TUNED)

cts_model = load_model(EXPORT_TLDR_CS_FINE_TUNED)
cts_model = lora_config_for(cts_model, EXPORT_TLDR_CS_FINE_TUNED, for_training=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
cts_outputs = batch_infer_from(cts_model, cts_tokenizer, generation_inputs, batch=4)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


5.94% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


11.88% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

83.17% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


89.11% complete


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


95.05% complete
100.00% complete


In [33]:
Path(EXPORT_CS_TLDR_RESULTS).write_text(json.dumps(cts_outputs))

2208586

## Conclusion

we have exported the results from the Model_2 as well. please refer to this notebook: [model_evaluation](https://github.com/au-nlp/project-milestone-p2-group-6/blob/main/lab/model_evaluation.ipynb)